In [43]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [45]:
tf.keras.backend.clear_session()

In [46]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 0
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [47]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.728082,0.294434,0.587982,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,30.109
1,2003-2,0.635571,0.728532,0.296211,0.588354,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,27.684
2,2003-3,0.635982,0.728982,0.297521,0.588725,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,26.534
3,2003-4,0.636393,0.729432,0.298766,0.589097,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,25.109
4,2003-5,0.636804,0.729882,0.299945,0.589469,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,NaN,NaN,0.587084,0.629024,NaN,24.501314,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,0.585891,0.629017,NaN,24.609100,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,0.584366,0.629010,NaN,24.538488,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.126
239,2022-12,NaN,NaN,0.582450,0.629003,NaN,24.688196,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.126


In [48]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.146764,-2.407288,-1.449375,-2.067761,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.113412,-2.364085,-1.423792,-2.040187,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.080060,-2.320881,-1.404924,-2.012614,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.046707,-2.277677,-1.386985,-1.985041,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.013355,-2.234474,-1.370013,-1.957467,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,0.868238,1.308074,1.131052,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.177023,0.846976,1.359203,1.127894,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.170724,0.825714,1.405650,1.124737,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.164425,0.804452,1.459075,1.121579,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [49]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      27.684
1      26.534
2      25.109
3      25.171
4      22.645
        ...  
236    51.171
237    49.126
238    49.126
239       NaN
240       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 241, dtype: float64

In [50]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.146764,-2.407288,-1.449375,-2.067761,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.113412,-2.364085,-1.423792,-2.040187,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.080060,-2.320881,-1.404924,-2.012614,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.046707,-2.277677,-1.386985,-1.985041,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.013355,-2.234474,-1.370013,-1.957467,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.324514,1.370260,1.121339,1.196483,1.186919,-1.069189,1.177341,-1.214407,1.097518,1.109211,...,1.272681,1.521503,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.323316,1.364348,1.103076,1.194330,1.173733,-1.161695,1.181165,-1.230415,1.105489,1.111823,...,1.288866,1.512161,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.322118,1.358436,1.083984,1.192177,1.160546,-1.230868,1.184990,-1.246424,1.113460,1.114436,...,1.304998,1.500026,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.320921,1.352525,1.070055,1.190024,1.147360,-1.298996,1.188814,-1.262433,1.121431,1.117048,...,1.332157,1.486995,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [51]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      27.684000
1      26.534000
2      25.109000
3      25.171000
4      22.645000
         ...    
163    42.491371
164    41.535029
165    40.578686
166    39.622343
167    38.666000
Name: Alagoas - Consumo de Cimento (t), Length: 168, dtype: float64

In [52]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [53]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, start_index)
reshaped_train.shape

(132, 36, 64)

In [54]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Alagoas - IDH,Alagoas - IDH Longevidade,Alagoas - value,Alagoas - IDH Renda,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
132,0.384263,0.339301,1.053656,0.606684,1.055214,1.307529,0.851394,-0.396992,0.813623,0.917513,...,0.467927,0.897908,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401
133,0.384465,0.335100,1.094151,0.604117,1.070281,1.261277,0.867621,-0.443166,0.829740,0.930914,...,0.505071,0.970274,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620
134,0.384667,0.330900,1.134973,0.601551,1.085348,1.213376,0.883848,-0.489340,0.845857,0.944315,...,0.541297,1.040830,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284
135,0.384868,0.326699,1.179185,0.598984,1.100415,1.163870,0.900074,-0.535515,0.861974,0.957716,...,0.571731,1.107516,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516
136,0.385070,0.322498,1.214733,0.596417,1.115482,1.113252,0.916301,-0.581689,0.878091,0.971117,...,0.611614,1.168977,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,0.868238,1.308074,1.131052,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.177023,0.846976,1.359203,1.127894,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.170724,0.825714,1.405650,1.124737,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.164425,0.804452,1.459075,1.121579,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [55]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 64)

In [56]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [57]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [58]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [59]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[167074060, 3534401086, 2225271934, 1541002260, 2540149802, 283400904, 1979713820, 202546564, 2208400269, 2418779846, 3968470979, 2081757771, 3439012279, 1520452588, 2323783480, 1415307746, 741003130, 2755562414, 1908810544, 605808405, 3964654352, 3851311338, 3144112721, 1243195414, 1335407739]


Step: 0 ___________________________________________
val_loss: 13.624161720275879
winner_seed: 167074060


Step: 1 ___________________________________________
val_loss: 9.126799583435059
winner_seed: 3534401086


Step: 2 ___________________________________________
val_loss: 9.57049560546875


Step: 3 ___________________________________________
val_loss: 10.38454818725586


Step: 4 ___________________________________________
val_loss: 13.245888710021973


Step: 5 ___________________________________________
val_loss: 8.304512977600098
winner_seed: 283400904


Step: 6 ___________________________________________
val_loss: 9.210493087768555


Step: 7 ___________________________________________
val_l

In [60]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 115ms/step - loss: 2026.8804 - val_loss: 234.7590
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 259.2535 - val_loss: 208.1586
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 300.3494 - val_loss: 389.3419
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 280.3024 - val_loss: 169.5230
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 153.8903 - val_loss: 422.6040
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 366.5293 - val_loss: 360.6728
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 499.7365 - val_loss: 209.5856
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 164.0509 - val_loss: 294.0760
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 276.9856 - val_loss: 65.2731
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - los

4/4 [==============================] - 0s 8ms/step - loss: 21.6043 - val_loss: 22.7553
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 21.7385 - val_loss: 37.6423
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 35.2871 - val_loss: 23.9941
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 23.0570 - val_loss: 23.2188
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.7464 - val_loss: 26.0678
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 25.6686 - val_loss: 22.9670
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.5234 - val_loss: 21.3604
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 23.3365 - val_loss: 26.7590
Epoch 89/10000
4/4 [==============================] - 0s 8ms/step - loss: 26.3119 - val_loss: 22.0352
Epoch 90/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.6839 - val_loss: 24.

Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 19.9588 - val_loss: 22.5985
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.2105 - val_loss: 20.9011
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.6467 - val_loss: 22.4936
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 24.2711 - val_loss: 23.7924
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.5727 - val_loss: 23.1685
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step - loss: 21.7622 - val_loss: 39.3619
Epoch 167/10000
4/4 [==============================] - 0s 8ms/step - loss: 27.9325 - val_loss: 20.7526
Epoch 168/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.8212 - val_loss: 19.9490
Epoch 169/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.6441 - val_loss: 21.1891
Epoch 170/10000
4/4 [==============================] - 0s 8ms/step - loss

4/4 [==============================] - 0s 7ms/step - loss: 20.2336 - val_loss: 19.1942
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.1173 - val_loss: 26.2621
Epoch 242/10000
4/4 [==============================] - 0s 8ms/step - loss: 23.6280 - val_loss: 21.7632
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.0108 - val_loss: 22.7846
Epoch 244/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.2400 - val_loss: 29.1409
Epoch 245/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.4481 - val_loss: 20.1777
Epoch 246/10000
4/4 [==============================] - 0s 8ms/step - loss: 21.2027 - val_loss: 33.8040
Epoch 247/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.3302 - val_loss: 19.7415
Epoch 248/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.1942 - val_loss: 22.6095
Epoch 249/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.9126 - val_

4/4 [==============================] - 0s 8ms/step - loss: 17.7666 - val_loss: 19.2089
Epoch 320/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.1634 - val_loss: 19.8900
Epoch 321/10000
4/4 [==============================] - 0s 8ms/step - loss: 17.9794 - val_loss: 21.8590
Epoch 322/10000
4/4 [==============================] - 0s 8ms/step - loss: 19.2029 - val_loss: 21.3682
Epoch 323/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.0520 - val_loss: 19.8436
Epoch 324/10000
4/4 [==============================] - 0s 8ms/step - loss: 19.0158 - val_loss: 19.4693
Epoch 325/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.9460 - val_loss: 20.0133
Epoch 326/10000
4/4 [==============================] - 0s 8ms/step - loss: 18.9399 - val_loss: 20.5914
Epoch 327/10000
4/4 [==============================] - 0s 8ms/step - loss: 19.2203 - val_loss: 22.1545
Epoch 328/10000
4/4 [==============================] - 0s 8ms/step - loss: 17.4511 - val_

...

4/4 [==============================] - 0s 8ms/step - loss: 7.6839 - val_loss: 14.1219
Epoch 2848/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.3253 - val_loss: 15.0686
Epoch 2849/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.7089 - val_loss: 13.2148
Epoch 2850/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.9916 - val_loss: 14.1070
Epoch 2851/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.4664 - val_loss: 15.9039
Epoch 2852/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.1047 - val_loss: 10.7492
Epoch 2853/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.3727 - val_loss: 13.6913
Epoch 2854/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.7071 - val_loss: 13.1417
Epoch 2855/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3440 - val_loss: 12.2023
Epoch 2856/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8037 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 7.4264 - val_loss: 18.1199
Epoch 2927/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.0873 - val_loss: 14.8420
Epoch 2928/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.8249 - val_loss: 14.9050
Epoch 2929/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.5527 - val_loss: 13.0862
Epoch 2930/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.8236 - val_loss: 16.4015
Epoch 2931/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.4864 - val_loss: 14.1570
Epoch 2932/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8807 - val_loss: 13.0907
Epoch 2933/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.1980 - val_loss: 12.4549
Epoch 2934/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7860 - val_loss: 11.2885
Epoch 2935/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2634 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 8.3187 - val_loss: 15.7025
Epoch 3006/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.9099 - val_loss: 16.1754
Epoch 3007/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.2599 - val_loss: 15.3215
Epoch 3008/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9175 - val_loss: 16.0505
Epoch 3009/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8496 - val_loss: 16.5470
Epoch 3010/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.6588 - val_loss: 14.2439
Epoch 3011/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.5121 - val_loss: 14.9131
Epoch 3012/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.4364 - val_loss: 12.6926
Epoch 3013/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0611 - val_loss: 12.7542
Epoch 3014/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1142 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 7.4346 - val_loss: 18.6344
Epoch 3085/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.7796 - val_loss: 16.4020
Epoch 3086/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.3593 - val_loss: 15.1299
Epoch 3087/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.4810 - val_loss: 17.8452
Epoch 3088/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.5275 - val_loss: 15.5671
Epoch 3089/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8998 - val_loss: 14.2065
Epoch 3090/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.5013 - val_loss: 14.8391
Epoch 3091/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.5643 - val_loss: 16.2692
Epoch 3092/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6088 - val_loss: 13.0455
Epoch 3093/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4695 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 6.0105 - val_loss: 17.9295
Epoch 3164/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9713 - val_loss: 19.6676
Epoch 3165/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5753 - val_loss: 18.9148
Epoch 3166/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1389 - val_loss: 19.9902
Epoch 3167/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7732 - val_loss: 15.0122
Epoch 3168/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4708 - val_loss: 14.5209
Epoch 3169/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2430 - val_loss: 12.2342
Epoch 3170/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.1199 - val_loss: 13.4582
Epoch 3171/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.6134 - val_loss: 17.4475
Epoch 3172/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.1595 - val_l

In [61]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [62]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 295ms/step
Month-1 - Error: [[7.480919]]
1/1 [==============================] - 0s 15ms/step
Month-2 - Error: [[2.1541252]]
1/1 [==============================] - 0s 14ms/step
Month-3 - Error: [[7.8101025]]
1/1 [==============================] - 0s 14ms/step
Month-4 - Error: [[11.285698]]
1/1 [==============================] - 0s 14ms/step
Month-5 - Error: [[17.164654]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[18.452442]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[12.923935]]
1/1 [==============================] - 0s 14ms/step
Month-8 - Error: [[11.692167]]
1/1 [==============================] - 0s 14ms/step
Month-9 - Error: [[9.816626]]
1/1 [==============================] - 0s 14ms/step
Month-10 - Error: [[10.764013]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[8.140829]]
1/1 [==============================] - 0s 15ms/step
Month-12 - Error: [[9.717356]]
1/1 

In [63]:
display(mae)
display(mape)

9.617

0.3467562

In [64]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [65]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[477.2106]] - Target[354.11599999999993]| =  Error: [[123.094604]]; MAPE:[[0.34761098]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[476.97763]] - Target[342.67499999999995]| =  Error: [[134.30264]]; MAPE:[[0.39192426]]
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[478.434]] - Target[406.294]| =  Error: [[72.139984]]; MAPE:[[0.17755611]]


[array([[123.094604]], dtype=float32),
 array([[134.30264]], dtype=float32),
 array([[72.139984]], dtype=float32)]

109.84574

0.3056971